In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/CutDebt/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(17015, 2)


,label,split_text
0,0,这笔 款项 也 可以 退还 。
1,1,最近 紧张
2,1,必须 是 今天 ？
3,0,谢谢 你
4,2,记得 吃饭
5,2,能 不能 少 点
6,0,我 的 妹妹 只 给 了 我 500 元
7,1,许多 事情 即将 到来 。
8,0,本金 我 能够 处理
9,0,我 知道


In [3]:
data.split_text.iloc[89]

'数额 太高   我 想 办法'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<17015x20934 sparse matrix of type '<class 'numpy.float64'>'
	with 122385 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.911254775198


In [6]:
print(clf.predict(phrase))

[0 1 1 ..., 2 0 0]


# Logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.87352336174


In [9]:
print(log_r.predict(phrase))

[0 1 0 ..., 0 0 0]


# Naive Bayes

In [10]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(phrase, data.label)
print(nb.score(phrase, data.label))

0.870349691449


In [11]:
print(nb.predict(phrase))

[0 1 0 ..., 2 0 0]


# LightGBM

In [12]:
# import lightgbm as lgb

In [13]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(data.label.values)
# onelabels = le.transform(data.label.values)

In [14]:
# onelabels

In [15]:
# from sklearn.preprocessing import MultiLabelBinarizer
# multicoder = MultiLabelBinarizer()
# lables = multicoder.fit_transform([data.label.values])

In [16]:
# lables

In [17]:
# data.label.values

In [18]:
# params = {
#     'learning_rate': 0.2,
#     'num_iterations':1000,
#     'application': 'multiclassova',
#     'num_class': 3,
#     'num_leaves': 31,
#     'verbosity': -1,
#     'metric': 'multi_error',
#     'data_random_seed': 2,
# #     'bagging_fraction': 0.8,
# #     'feature_fraction': 0.6,
#     'nthread': 4,
#     'lambda_l1': 1,
#     'lambda_l2': 1,
#     'early_stopping_round':200
# } 

# # lgbm_train = lgb.Dataset(phrase, data.label)
# lgbm_train = lgb.Dataset(phrase, onelabels)
# lgbm_val = lgb.Dataset(phrase, onelabels)
# lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

In [19]:
# print(lgbm_model.predict(phrase))

# Save

In [20]:
# import pickle

# # save tfidf
# pickle.dump(phrase_vectorizer, open("../../savedModel/CutDebt/tfidf.pickle", "wb"))
# # pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# # pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# # save linear svc
# pickle.dump(clf, open("../../savedModel/CutDebt/LinearSVC.pickle", "wb"))
# # save logistic
# pickle.dump(log_r, open("../../savedModel/CutDebt/Logistic.pickle", "wb"))
# # save lightGBM
# pickle.dump(lgbm_model, open("../../savedModel/CutDebt/Lgbm.pickle", "wb"))

# test

In [31]:
import jieba
sentence = '好'
sentence = jieba.cut(sentence, cut_all = False)
sentence = ' '.join(sentence)
test = phrase_vectorizer.transform([sentence])

In [32]:
clf.predict_proba(test) # linear svc

array([[ 0.90468671,  0.03936992,  0.05594337]])

In [33]:
log_r.predict_proba(test) # logistic

array([[ 0.88508681,  0.03186908,  0.08304411]])

In [34]:
nb.predict_proba(test)

array([[ 0.89450399,  0.03005283,  0.07544318]])

In [30]:
# lgbm_model.predict(test) # light gbm

# Labeling

In [23]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.983986058554


# Save the whole thing

In [24]:
# import jieba
# import numpy as np

# class CutDebt:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred)<threshold:
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [25]:
%load_ext autoreload
%autoreload 2

In [1]:
import pickle
clf = pickle.load(open("../../savedModel/CutDebt/LinearSVC.pickle", "rb"))
lgbm_model = pickle.load(open("../../savedModel/CutDebt/Lgbm.pickle", "rb"))
phrase_vectorizer = pickle.load(open("../../savedModel/CutDebt/tfidf.pickle", "rb"))
log_r = pickle.load(open("../../savedModel/CutDebt/Logistic.pickle", "rb"))

In [2]:
from CutDebt_py import CutDebt
cutd = CutDebt(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer,) #jieba_path='../WordCut/userdict.txt')

pickle.dump(cutd, open("../../savedModel/CutDebt/CutDebt.pickle", "wb"))

In [4]:
cutd.classify('m')

(0, [0, 0.53057567729860577])